In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from scipy.stats import kstest
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# 8.1

In [5]:
q1 = pd.read_csv(r'C:\Users\20123\Desktop\数学\时间序列分析\lab4/data/m-mrk2vw.txt', sep='\s+')


In [13]:
q1

,date,MRK,JNJ,GE,GM,F,VW
0,19600129,-7.745788,-4.868722,-14.786564,-12.960767,-13.392461,-6.865793
1,19600229,7.590400,3.201985,4.572261,-2.914152,-1.299710,1.410210
2,19600331,0.651076,-5.641375,0.972951,-3.315252,-11.037529,-1.293024
3,19600429,5.439443,-0.447399,-1.400562,-0.563384,-5.536473,-1.544872
4,19600531,11.647462,17.255822,0.842739,-0.566602,0.718513,3.341737
...,...,...,...,...,...,...,...
583,20080829,8.083685,3.473374,-0.673865,-10.165406,-7.346679,1.098346
584,20080930,-11.045123,-1.646277,-8.500793,-5.657035,15.350962,-10.320728
585,20081031,-1.951723,-12.172366,-26.775135,-48.988312,-86.475672,-20.422489
586,20081128,-14.696059,-3.821706,-12.776406,-9.981039,20.563924,-8.905091


## 8.1(a)

In [15]:
names = q1.columns
data = q1.copy(deep=True)
data =  data.drop(names[0], axis=1) 
data

,MRK,JNJ,GE,GM,F,VW
0,-7.745788,-4.868722,-14.786564,-12.960767,-13.392461,-6.865793
1,7.590400,3.201985,4.572261,-2.914152,-1.299710,1.410210
2,0.651076,-5.641375,0.972951,-3.315252,-11.037529,-1.293024
3,5.439443,-0.447399,-1.400562,-0.563384,-5.536473,-1.544872
4,11.647462,17.255822,0.842739,-0.566602,0.718513,3.341737
...,...,...,...,...,...,...
583,8.083685,3.473374,-0.673865,-10.165406,-7.346679,1.098346
584,-11.045123,-1.646277,-8.500793,-5.657035,15.350962,-10.320728
585,-1.951723,-12.172366,-26.775135,-48.988312,-86.475672,-20.422489
586,-14.696059,-3.821706,-12.776406,-9.981039,20.563924,-8.905091


In [18]:
data.mean(), data.cov(), data.corr()

(MRK    0.966503
 JNJ    1.169972
 GE     0.720937
 GM     0.124346
 F      0.448921
 VW     0.724589
 dtype: float64,
            MRK        JNJ         GE         GM          F         VW
 MRK  49.579144  23.867654  17.925271  10.228013  12.691428  14.298334
 JNJ  23.867654  39.590745  17.426005   8.818861  11.854873  14.247467
 GE   17.925271  17.426005  41.373907  23.551507  25.223376  20.489913
 GM   10.228013   8.818861  23.551507  72.819340  54.193257  20.073579
 F    12.691428  11.854873  25.223376  54.193257  85.298190  21.853524
 VW   14.298334  14.247467  20.489913  20.073579  21.853524  19.938925,
           MRK       JNJ        GE        GM         F        VW
 MRK  1.000000  0.538720  0.395779  0.170223  0.195160  0.454763
 JNJ  0.538720  1.000000  0.430564  0.164245  0.204000  0.507096
 GE   0.395779  0.430564  1.000000  0.429074  0.424590  0.713388
 GM   0.170223  0.164245  0.429074  1.000000  0.687626  0.526806
 F    0.195160  0.204000  0.424590  0.687626  1.000000  0.

In [42]:
## 滞后1阶
data0 = np.matrix(data.tail(len(data)-1))
datal = np.matrix(data.head(len(data)-1))
# sm.tsa.stattools.ccf(data0, datal, adjusted=True, fft=True)
m = np.matrix(data.mean())
t = len(data)
gamma = 0
for i in range(t-1):
    gamma = gamma + 1/t*(((data0[i]-m).T)*(datal[i]-m))
d = np.diag(np.diagonal(np.matrix(data.cov()**0.5)))
rho = np.linalg.inv(d)*gamma*np.linalg.inv(d)
print(rho)

[[-0.0753788   0.00385564  0.02305156  0.0069848   0.04071037  0.0077513 ]
 [-0.00901875 -0.01420992 -0.00446422 -0.05018823 -0.04989477 -0.05088971]
 [ 0.00830133 -0.00870017  0.01266336  0.01600492  0.02887132  0.06479838]
 [-0.01899133 -0.00858122  0.15912774  0.08231161  0.04508453  0.16413495]
 [ 0.00474495  0.03559397  0.07040303  0.02846709 -0.05659173  0.10363405]
 [ 0.02407538  0.06107815  0.06960957  0.03946015  0.06247645  0.09983266]]


根据书中定义，$\frac{2}{\sqrt{T}}$为5%渐近水平的临界值，即0.04123930494211612，可以看出延迟1的时候有边际自相关性，对于延迟23456同理可求解如下：

In [78]:
data0 = []
datai = []
for i in range(2,7):
    data0.append(np.matrix(data.tail(len(data)-i))) 
    datai.append(np.matrix(data.head(len(data)-i)))
gammai = [0,0,0,0,0] 
for i in range(5):
    for j in range(t-2-i):
        gammai[i] = gammai[i] + 1/t*(((data0[i][j]-m).T)*(datai[i][j]-m))
rhoi = []
for i in range(5):
    rhoi.append(np.linalg.inv(d)*gammai[i]*np.linalg.inv(d))
tes = []
for i in range(5):
    judge = (np.abs(np.array(rhoi[i])) > 2/t**0.5)
    l = True
    for j in judge:
        for k in j:
            if k==True:
                l = False
                break
    tes.append(l)
tes

[True, False, True, False, False]

除了2，4延迟，其他全都拒绝原假设，即有边际自相关性。

## 8.1（c）

由（b），有引导-延迟关系。

# 8.2

In [76]:
q2 = pd.read_csv(r'C:\Users\20123\Desktop\数学\时间序列分析\lab4/data/m-gs1n10.txt', sep='\s+')

In [77]:
q2

,Year,Mon,Day,gs1,gs10
0,1953,4,1,2.36,2.83
1,1953,5,1,2.48,3.05
2,1953,6,1,2.45,3.11
3,1953,7,1,2.38,2.93
4,1953,8,1,2.28,2.95
...,...,...,...,...,...
674,2009,6,1,0.51,3.72
675,2009,7,1,0.48,3.56
676,2009,8,1,0.46,3.59
677,2009,9,1,0.40,3.40


## 8.2(a)

In [81]:
data = q2[['gs1','gs10']]
c = data.diff().dropna()
c

,gs1,gs10
1,0.12,0.22
2,-0.03,0.06
3,-0.07,-0.18
4,-0.10,0.02
5,-0.08,-0.08
...,...,...
674,0.01,0.43
675,-0.03,-0.16
676,-0.02,0.03
677,-0.06,-0.19


In [83]:
from statsmodels.tsa.vector_ar.var_model import VAR
mod = VAR(c)
lag_order = mod.select_order()
print(lag_order.summary())

 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -5.183      -5.169    0.005612      -5.178
1       -5.381      -5.340    0.004602      -5.365
2       -5.462     -5.393*    0.004247      -5.435
3       -5.475      -5.380    0.004189      -5.438
4       -5.481      -5.359    0.004164      -5.434
5       -5.482      -5.332    0.004161      -5.424
6       -5.542      -5.365    0.003920     -5.473*
7       -5.537      -5.333    0.003938      -5.458
8       -5.547      -5.315    0.003901      -5.457
9       -5.540      -5.281    0.003928      -5.439
10      -5.540      -5.253    0.003928      -5.429
11      -5.551      -5.237    0.003885      -5.429
12      -5.566      -5.225    0.003828      -5.433
13      -5.576      -5.208    0.003789      -5.433
14      -5.578      -5.183    0.003781      -5.425
15      -5.579      -5.156    0.003778      -5.415
16      -5.593      -5.143    0

d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [85]:
## 由BIC准则，选VAR（2）
res = mod.fit(2)
print(res.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 28, May, 2022
Time:                     20:27:51
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -5.43675
Nobs:                     676.000    HQIC:                  -5.47769
Log likelihood:          -48.2018    FPE:                 0.00407225
AIC:                     -5.50356    Det(Omega_mle):      0.00401267
--------------------------------------------------------------------
Results for equation gs1
             coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------
const          -0.002297         0.015035           -0.153           0.879
L1.gs1          0.308134         0.060446            5.098           0.000
L1.gs10         0.296775         0.091765            3.234           0.001
L2.gs1     

In [90]:
res.params

,gs1,gs10
const,-0.002297,0.000672
L1.gs1,0.308134,-0.000937
L1.gs10,0.296775,0.377486
L2.gs1,-0.103923,0.048057
L2.gs10,-0.246563,-0.285432


说明当期的差分值与上一期的差分值成正相关，与上两期的差分值成负相关，且两个序列之间具有引导延迟关系。
$$
\left(\begin{array}{l}
1-0.308 B+0.104 B^{2} \\
1-0.297 B+0.247 B^{2}
\end{array}\right)\left(\begin{array}{l}
r_{1 t} \\
r_{2 t}
\end{array}\right)=\left(\begin{array}{c}
-0.002 \\
0.001
\end{array}\right)+\left(\begin{array}{l}
e_{1t} \\
e_{2t}
\end{array}\right)
$$

## 8.2(b)

In [96]:
from statsmodels.tsa.statespace.varmax import VARMAX
b = []
for i in range(1,5):
    mod = VARMAX(c,order=(0,i))
    res = mod.fit()
    b.append(res.bic)
# print(res.summary())

d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
d:\Anaco

In [100]:
print( b.index(min(b))+1)

1


In [102]:
mod = VARMAX(c,order=(0,1))
res = mod.fit()
res.summary()

d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Dep. Variable:,"['gs1', 'gs10']",No. Observations:,678
Model:,VMA(1),Log Likelihood,-48.983
,+ intercept,AIC,115.965
Date:,"Sat, 28 May 2022",BIC,156.637
Time:,20:54:43,HQIC,131.711
Sample:,0,,
,- 678,,
Covariance Type:,opg,,
Ljung-Box (L1) (Q):,"0.41, 1.89",Jarque-Bera (JB):,"3905.37, 115.34"
Prob(Q):,"0.52, 0.17",Prob(JB):,"0.00, 0.00"
Heteroskedasticity (H):,"0.85, 2.66",Skew:,"-1.09, -0.17"


In [103]:
res.params

intercept.gs1       -0.002778
intercept.gs10       0.000993
L1.e(gs1).gs1        0.358465
L1.e(gs10).gs1       0.293885
L1.e(gs1).gs10      -0.014753
L1.e(gs10).gs10      0.456954
sqrt.var.gs1         0.388292
sqrt.cov.gs1.gs10    0.200440
sqrt.var.gs10        0.162024
dtype: float64

$$
\left(\begin{array}{l}
r_{1 t} \\
r_{2 t}
\end{array}\right)=\left(\begin{array}{c}
-0.002778 \\
0.000993
\end{array}\right)+\left(\begin{array}{l}
e_{1t} \\
e_{2t}
\end{array}\right)+\left(\begin{array}{l}
-0.358465 &-0.293885  \\
0.014753 &-0.456954 
\end{array}\right)\left(\begin{array}{l}
e_{1,t-1} \\
e_{2,t-1}
\end{array}\right)
$$
说明两个序列之间具有引导延迟关系。
两个模型拟合都不错。

# 8.3

In [105]:
from math import log
lg = q2[['gs1','gs10']].copy(deep=True)
name = lg.columns
for i in range (2):
    lg.loc[:,name[i]] = lg.loc[:,name[i]].apply(lambda x: log(1+x))

In [108]:
lg

,gs1,gs10
0,1.211941,1.342865
1,1.247032,1.398717
2,1.238374,1.413423
3,1.217876,1.368639
4,1.187843,1.373716
...,...,...
674,0.412110,1.551809
675,0.392042,1.517323
676,0.378436,1.523880
677,0.336472,1.481605


In [112]:
b = []
for i in range(1,5):
    for j in range(1,5):
        mod = VARMAX(lg,order=(i,j))
        res = mod.fit()
        b.append(res.bic)

d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: Estimation

In [113]:
print( b.index(min(b)))

4


VARMA(2,1)

In [115]:
mod = VARMAX(lg,order=(2,1))
res = mod.fit()
res.summary()

d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)
d:\Anaconda\envs\tf_torch\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Dep. Variable:,"['gs1', 'gs10']",No. Observations:,679
Model:,"VARMA(2,1)",Log Likelihood,2672.779
,+ intercept,AIC,-5311.557
Date:,"Sat, 28 May 2022",BIC,-5234.707
Time:,21:28:15,HQIC,-5281.808
Sample:,0,,
,- 679,,
Covariance Type:,opg,,
Ljung-Box (L1) (Q):,"0.58, 0.05",Jarque-Bera (JB):,"251.71, 410.42"
Prob(Q):,"0.45, 0.82",Prob(JB):,"0.00, 0.00"
Heteroskedasticity (H):,"1.06, 2.20",Skew:,"-0.48, 0.22"


可以利用所建立的ARMA（2,1），利用历史对数数据进行对于未来的预测。